### Initialise data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.lines as mlines
from defdap.quat import Quat
import defdap.ebsd as ebsd
import defdap.hrdic as hrdic
from skimage import transform as tf
from skimage import io, measure
from IPython.display import display
import scipy as scipy
import peakutils

import os
if os.name == 'posix':
    %matplotlib osx

#Load in DIC maps
DicFilePath = "/Users/mbcx9rt5/Desktop/"
DicMap = hrdic.Map(DicFilePath, "irr.TXT")
DicMap.setPatternPath("irr_000_8bin.jpg", 2)
DicMap.setCrop(xMin=150,xMax=293,yMin=115,yMax=214)

DicMap2 = hrdic.Map(DicFilePath, "nonirr.TXT")
DicMap2.setPatternPath("nonirr_000_8bin.jpg", 2)
DicMap2.setCrop(xMin=131,xMax=166,yMin=100,yMax=99)

pixelinm=78.125e-9

#Load in EBSD map and calculate misorientation
EbsdMap = ebsd.Map("/Users/mbcx9rt5/Desktop/irr", "hexagonal")
EbsdMap.loadSlipSystems("defdap/slip_systems/hexagonal_withca.txt", cOverA=1.593)
EbsdMap.buildQuatArray()
EbsdMap.findBoundaries(boundDef = 3.6)
#EbsdMap.boundaries = scipy.misc.imread('newdata/irr_boundaries.bmp').astype(int)
#EbsdMap.boundaries[EbsdMap.boundaries == 0] = -1
#EbsdMap.boundaries[EbsdMap.boundaries == 255] = 0
EbsdMap.findGrains(minGrainSize= 100)
EbsdMap.calcGrainMisOri(calcAxis = False)
EbsdMap.calcAverageGrainSchmidFactors(loadVector=np.array([1, 0, 0]))

EbsdMap2 = ebsd.Map("/Users/mbcx9rt5/Desktop/ni", "hexagonal")
EbsdMap2.loadSlipSystems("defdap/slip_systems/hexagonal_withca.txt", cOverA=1.593)
EbsdMap2.buildQuatArray()
EbsdMap2.findBoundaries(boundDef = 3.6)
#EbsdMap2.boundaries = scipy.misc.imread('newdata/nonirr_boundaries.bmp').astype(int)
#EbsdMap2.boundaries[EbsdMap2.boundaries == 0] = -1
#EbsdMap2.boundaries[EbsdMap2.boundaries == 255] = 0
EbsdMap2.findGrains(minGrainSize= 100)
EbsdMap2.calcGrainMisOri(calcAxis = False)
EbsdMap2.calcAverageGrainSchmidFactors(loadVector=np.array([1, 0, 0]))

DicMap2.homogPoints = np.array((((326, 377), (118 , 540),(1166, 955), (1108, 1012), (142, 1042), (252, 1092), (64, 766), (68,200), 
                                 (710, 936), (580, 1128), (1026, 484), (1204, 306), (1166, 600), (1044,  852), (888, 1101), (636, 609),
                                 (485, 890), (486, 628), (700, 276), (414, 219), (874, 670), (863, 59), (226, 44), (552, 28))))
EbsdMap2.homogPoints = np.array(((167, 232), (80, 286), (484, 469), (462, 489), (79, 472), (121, 494), (54, 368), (67, 160), 
                                 (305, 446), (253, 516), (440, 289), (513, 229), (491, 337), (438, 428), (373, 519), (281, 325),
                                 (218, 423), (226, 327), (317, 204), (203, 174), (378, 353), (385, 128), (135, 106), (262, 106)))
DicMap.homogPoints = np.array(( (430, 219), (107, 1082), (111, 164), (547, 1106), (1164, 1108),
                               (1410, 887), (1424, 608), (1290, 180), (846, 236), (814, 36),
                               (462, 58), (1000, 620), (43, 652), (206, 498), (476, 723),
                               (582, 747), (850, 1088), (1183, 764), (686, 428), (668, 538),
                               (854, 851) ))
EbsdMap.homogPoints = np.array(( (202, 141), (57, 434), (78, 118), (228, 450), (471, 461), (570, 388),
                                (582, 290), (539, 144), (365, 155), (358, 87), (217, 90), (415, 288),
                                (40, 284), (107, 233), (209, 315), (247, 325), (347, 447), (484, 341),
                                (299, 215), (289, 253), (346, 357) ))

DicMap.homogPoints[:,:1] -= 50
DicMap.homogPoints[:,1:2] -= 15

DicMap.linkEbsdMap(EbsdMap)
DicMap.findGrains(minGrainSize=100)

DicMap2.linkEbsdMap(EbsdMap2)
DicMap2.findGrains(minGrainSize=100)

In [ ]:
#EXPORT BOUNDARY TO FILE
from skimage import morphology as mph
import matplotlib as mpl
cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['white', 'black'], 256)
cmap1._init()
cmap1._lut[:, -1] = np.linspace(0, 1, cmap1.N + 3)
boundariesImage = -DicMap2.boundaries
boundariesImage2 = mph.binary_dilation(boundariesImage)
plt.imshow(boundariesImage2, cmap=cmap1, vmin=0, vmax=1)

import scipy.misc
scipy.misc.imsave('newdata/nirr_exp.bmp', boundariesImage)

In [ ]:
#DicMap2.homogPoints = np.array((   ))
#EbsdMap2.homogPoints = np.array((   ))

#DicMap2.setHomogPoint(display="pattern")
#EbsdMap2.setHomogPoint()

DicMap2.setHomogPoint()
EbsdMap2.setHomogPoint()

### EBSD Misori plot

In [ ]:
EbsdMap.calcGrainMisOri(calcAxis = True)
EbsdMap2.calcGrainMisOri(calcAxis = True)

In [ ]:
EbsdMap2.plotMisOriMap(plotGBs=True, boundaryColour='black', vmin=-4, vmax=4,
                      cmap="coolwarm")

### Plot DIC maps

In [2]:
DicMap2.plotMaxShear(plotGBs=True,vmin=0,vmax=0.08,plotColourBar=True,dilateBoundaries=True)

### Plot EBSD Maps

In [ ]:
EbsdMap2.plotMisOriMap(plotGBs=True,vmin=0,vmax=6)

### Stats

In [ ]:
DicMap.printStatsTable(percentiles=['Min', 1, 5, 'Mean', 95, 99.99, 'Max'], components=['f11','mss'])
DicMap2.printStatsTable(percentiles=['Min', 1, 5, 'Mean', 95, 99.99, 'Max'], components=['f11','mss'])

#### loglog

In [ ]:
irr_crop= DicMap.crop(DicMap.max_shear)
irr_crop= DicMap.crop(DicMap.max_shear) / np.mean(DicMap.crop(DicMap.max_shear))
ni_crop= DicMap2.crop(DicMap2.max_shear)
ni_crop= DicMap2.crop(DicMap2.max_shear) / np.mean(DicMap2.crop(DicMap2.max_shear))

irr2=[item for sublist in irr_crop for item in sublist]
nirr2=[item for sublist in ni_crop for item in sublist]

f, (ax1) = plt.subplots(1, figsize=(6, 4))

counts, bins, bars = ax1.hist(irr2, bins=np.logspace(np.log10(0.001),np.log10(50.0),100), color='r', histtype='step',  label='Irradiated',normed=True)
plt.xlabel("Effective shear strain (multiples of average)", fontsize="12")
plt.ylabel("Frequency", fontsize="12")
center = (bins[:-1] + bins[1:]) / 2
np.savetxt('irr.csv', (center, counts), delimiter='\n')

counts2, bins, bars = ax1.hist(nirr2, bins=np.logspace(np.log10(0.001),np.log10(50.0),100), color='g', histtype='step',  label='Non irradiated',normed=True)
plt.xscale("log", nonposx='clip'); plt.yscale("log", nonposy='clip');
plt.xlim(1e-2,5e1); 
plt.ylim(0,1e2);

center = (bins[:-1] + bins[1:]) / 2
np.savetxt('nonirr.csv', (center, counts2), delimiter='\n')

plt.tick_params(top='on', right='on', which='both', direction='in')
plt.tick_params(axis='y', which='minor')
plt.axvline(1, color='black', linestyle='dashed', linewidth=1)
plt.legend()
f.tight_layout()
plt.show()

f.savefig('histograms.png',dpi=400, pad_inches=0)

### Schmid factor map

In [ ]:
import matplotlib as mpl
from skimage import morphology as mph

selDicMap=DicMap
percentile=95
fig = plt.figure(frameon=False, figsize=(8,7))
schmidlist=[];

ax = plt.Axes(fig, [0., 0., 0.9, 1.])
ax.set_axis_off()
fig.add_axes(ax)

cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['black', 'black'], 256)
cmap1._init()
cmap1._lut[:, -1] = np.linspace(0, 1, cmap1.N + 3)
cmap2 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['red', 'red'], 256)
cmap2._init()
cmap2._lut[:, -1] = np.linspace(0, 1, cmap2.N + 3)

avschmid = np.zeros([selDicMap.yDim, selDicMap.xDim])
for Grain in selDicMap.grainList:
    #max Schmid factor
    currentSchmidFactor = np.max([Grain.ebsdGrain.averageSchmidFactors[1],Grain.ebsdGrain.averageSchmidFactors[2],Grain.ebsdGrain.averageSchmidFactors[3]])
    schmidlist.append(currentSchmidFactor)       
    # currentSchmidFactor = grain.averageSchmidFactors[0][0]
    for coord in Grain.coordList:
        avschmid[coord[1], coord[0]] = currentSchmidFactor
prism=ax.imshow(avschmid,interpolation='none', cmap='gray', vmin=0, vmax=0.5)

boundariesImage = -selDicMap.boundaries
boundariesImage2 = mph.binary_dilation(boundariesImage)
img=ax.imshow(boundariesImage2, cmap=cmap1, vmin=0, vmax=1)

cbar=plt.colorbar(prism, ax=ax,fraction=0.03, pad=0.02, anchor=(0.0, 0.12))
cbar.set_label(label="Prismatic Schmid Factor",size=12)
cbar.ax.tick_params(labelsize=11) 

threshold=np.percentile(selDicMap.crop(selDicMap.max_shear),percentile)
print(threshold)
thresholdmap=selDicMap.crop(selDicMap.max_shear)>threshold
ax.imshow(thresholdmap,cmap=cmap2,alpha=0.9)

ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)

fig.savefig('fig2.png',dpi=600, bbox_inches='tight', pad_inches=0)
print('Average prismatic schmid: {0:.3f} +- {1:.3f}'.format(np.mean(schmidlist),np.std(schmidlist)))